In [25]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torch import optim
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image
import os
import numpy as np
from torchvision import transforms

In [26]:
class ParkingLotDataset(Dataset):
    def __init__(self, batch_folder, labels_file):
        """
        Initialize the ParkingLotDataset with lazy loading of batches.

        Args:
        - batch_folder (str): Path to the folder containing .npy batch files.
        - labels_file (str): Path to the file containing the labels.
        """
        self.batch_folder = batch_folder

        # Load labels
        with open(labels_file, 'r') as f:
            self.labels = [int(line.split(":")[-1].strip()) for line in f]

        # Get the list of .npy files
        self.batch_files = sorted(os.listdir(batch_folder))

        # Map labels to batches (assuming labels are in order)
        self.num_batches = len(self.batch_files)
        self.batch_indices = []  # Maps global index to (batch_file, batch_idx)
        for i, batch_file in enumerate(self.batch_files):
            batch_path = os.path.join(batch_folder, batch_file)
            batch_data = np.load(batch_path)
            for idx in range(len(batch_data)):
                self.batch_indices.append((batch_file, idx))

        # Check alignment of images and labels
        assert len(self.batch_indices) == len(self.labels), (
            "Mismatch between the number of images and labels!"
        )

    def __len__(self):
        """Returns the total number of samples."""
        return len(self.labels)

    def __getitem__(self, idx):
        batch_file, batch_idx = self.batch_indices[idx]
        batch_path = os.path.join(self.batch_folder, batch_file)
        batch_data = np.load(batch_path)  # Load the specific batch
        image = batch_data[batch_idx]  # Get the specific image within the batch
        label = self.labels[idx]  # Get the corresponding label
        
        # Transpose image from (H, W, C) to (C, H, W) for PyTorch
        image = image.transpose((2, 0, 1))
        
        return torch.tensor(image, dtype=torch.float32), label

In [27]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # First Convolutional Block
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: [32, 256, 256]
        self.dropout1 = nn.Dropout(0.25)

        # Second Convolutional Block
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: [64, 128, 128]
        self.dropout2 = nn.Dropout(0.25)

        # Third Convolutional Block
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: [128, 64, 64]
        self.dropout3 = nn.Dropout(0.25)

        # Fourth Convolutional Block (optional)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.relu4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: [256, 32, 32]
        self.dropout4 = nn.Dropout(0.25)

        # Fully Connected Layers
        self.fc1 = nn.Linear(256 * 32 * 32, 128)  # Adjust input size
        self.relu5 = nn.ReLU()
        self.dropout5 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 1)  # Single output for regression

    def forward(self, x):
        x = self.pool1(self.relu1(self.bn1(self.conv1(x))))
        x = self.dropout1(x)
        x = self.pool2(self.relu2(self.bn2(self.conv2(x))))
        x = self.dropout2(x)
        x = self.pool3(self.relu3(self.bn3(self.conv3(x))))
        x = self.dropout3(x)
        x = self.pool4(self.relu4(self.bn4(self.conv4(x))))  # Optional
        x = self.dropout4(x)
        x = x.view(x.size(0), -1)  # Flatten
        # print(f"Flattened shape: {x.shape}")  # Debugging
        x = self.fc1(x)
        x = self.relu5(x)
        x = self.fc2(x)
        return x

In [28]:
image_folder = "/Users/aslandalhoffbehbahani/Documents/02461_Exam_Project/Processed"
labels_file = "/Users/aslandalhoffbehbahani/Documents/02461_Exam_Project/Labels_Num.txt"

labels = []

with open("/Users/aslandalhoffbehbahani/Documents/02461_Exam_Project/Labels_Num.txt", "r") as file:
    for line in file:
        # Split the line by ":" and strip whitespace
        label = line.split(":")[-1].strip()
        # Convert the label to an integer
        labels.append(int(label))



In [29]:
transform = transforms.ToTensor()

# Create dataset
dataset = ParkingLotDataset(image_folder, labels_file)
limited_dataset = torch.utils.data.Subset(dataset, range(1000))

# Create data loader
# train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

total_size = len(dataset)
train_size = int(0.7 * total_size)  # 70% for training
val_size = int(0.15 * total_size)  # 15% for validation
test_size = total_size - train_size - val_size  # 15% for testing

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [30]:
# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
criterion = nn.MSELoss()  # Regression task -> Mean Squared Error loss
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
print("Step 1 done")

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    print(f"Epoch {epoch+1}/{num_epochs}")
    
    for batch_idx, (images, labels) in enumerate(train_loader, start=1):
        images, labels = images.to(device), labels.to(device, dtype=torch.float32)
        print(f"Batch {batch_idx}/{len(train_loader)}", end="\r")
        # print(f"Image batch shape: {images.shape}")  # Debugging
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        outputs = outputs.squeeze()  # Remove unnecessary dimensions
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}")
    
    # Validation phase
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device, dtype=torch.float32)
            outputs = model(images).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    
    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}")

# Save the model
torch.save(model.state_dict(), "car_counting_cnn.pth")
print("Model saved!")

# Test phase
model.eval()
test_loss = 0.0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device, dtype=torch.float32)
        outputs = model(images).squeeze()
        loss = criterion(outputs, labels)
        test_loss += loss.item()

avg_test_loss = test_loss / len(test_loader)
print(f"Test Loss: {avg_test_loss:.4f}")

Step 1 done
Epoch 1/10
Epoch [1/10], Training Loss: 2719.2394
Epoch [1/10], Validation Loss: 1533.1974
Epoch 2/10
Epoch [2/10], Training Loss: 1500.3756
Epoch [2/10], Validation Loss: 1522.7801
Epoch 3/10
Epoch [3/10], Training Loss: 1488.6417
Epoch [3/10], Validation Loss: 1510.9907
Epoch 4/10
Epoch [4/10], Training Loss: 1477.0986
Epoch [4/10], Validation Loss: 1498.5856
Epoch 5/10
Epoch [5/10], Training Loss: 1464.6872
Epoch [5/10], Validation Loss: 1485.8445
Epoch 6/10
Epoch [6/10], Training Loss: 1452.1632
Epoch [6/10], Validation Loss: 1472.9582
Epoch 7/10
Epoch [7/10], Training Loss: 1440.3469
Epoch [7/10], Validation Loss: 1460.0358
Epoch 8/10
Epoch [8/10], Training Loss: 1426.7862
Epoch [8/10], Validation Loss: 1447.1349
Epoch 9/10
Epoch [9/10], Training Loss: 1414.8759
Epoch [9/10], Validation Loss: 1434.2589
Epoch 10/10
Epoch [10/10], Training Loss: 1400.9256
Epoch [10/10], Validation Loss: 1421.4818
Model saved!
Test Loss: 1408.7228
